In [1]:
import os
import sys
import torch
from pathlib import Path

# Define the repository path
repo_path = Path(r"C:\Users\abhin\Desktop\computer_vision\face_detection_using_yolo5s\yolov5-face")

# ✅ Check if the repository path exists
if not repo_path.exists():
    print(f"❌ Error: Repository path '{repo_path}' does not exist. Check the path and try again.")
    sys.exit(1)

# ✅ Change the working directory
try:
    os.chdir(repo_path)
    print(f"✅ Changed working directory to: {os.getcwd()}")
except Exception as e:
    print(f"❌ Error changing directory: {e}")
    sys.exit(1)

# ✅ Add repo path to sys.path if not already added
if str(repo_path) not in sys.path:
    sys.path.append(str(repo_path))
    print(f"✅ Added '{repo_path}' to Python sys.path")

# ✅ Verify if 'models.experimental' can be imported
try:
    from models.experimental import attempt_load
    print("✅ Successfully imported 'models.experimental'.")
except ModuleNotFoundError:
    print("❌ Error: Could not find 'models.experimental'. Ensure the YOLOv5-Face repo is correctly cloned and accessible.")
    sys.exit(1)

# ✅ Define the model path (NOW as a Path object)
model_path = Path(r"C:\Users\abhin\Desktop\computer_vision\face_detection_using_yolo5s\yolov5s-face.pt")

# ✅ Check if the model file exists
if not model_path.exists():  # ✅ Now it works correctly!
    print(f"❌ Error: Model file '{model_path}' not found. Please verify the file path.")
    sys.exit(1)

# ✅ Select device (CUDA or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"⚙️ Using device: {device}")

# ✅ Load the YOLOv5-Face model
try:
    model = attempt_load(str(model_path), map_location=device)  # Ensure model path is string for attempt_load
    model.eval()
    print("✅ Model loaded successfully!")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    sys.exit(1)


✅ Changed working directory to: C:\Users\abhin\Desktop\computer_vision\face_detection_using_yolo5s\yolov5-face
✅ Added 'C:\Users\abhin\Desktop\computer_vision\face_detection_using_yolo5s\yolov5-face' to Python sys.path
✅ Successfully imported 'models.experimental'.
⚙️ Using device: cpu


C:\Users\abhin\Desktop\computer_vision\face_detection_using_yolo5s\yolov5-face\models\experimental.py:118: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.append(torch.l

Fusing layers... 
✅ Model loaded successfully!


c:\Users\abhin\Desktop\.venv\Lib\site-packages\torch\functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3596.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [2]:
import torch
import cv2
import numpy as np


In [3]:
CONF_THRESH = 0.7
NMS_THRESH = 0.3

In [4]:
cap = cv2.VideoCapture(0)

In [5]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert image to tensor
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (320, 320))  # Resize for YOLOv5 input
    img_tensor = torch.from_numpy(img_resized).float() / 255.0
    img_tensor = img_tensor.permute(2, 0, 1).unsqueeze(0)  # Shape: (1, 3, 320, 320)

    # Run inference
    with torch.no_grad():
        pred = model(img_tensor)  # Output is a tuple
        pred = pred[0]  # Extract first element from tuple (this is our detections)

    # Convert detections to NumPy
    pred = pred.cpu().numpy()

    # print("Shape of pred:", pred.shape)
    # print("Content of pred:", pred)

    boxes = []
    confidences = []

    # Process detections
    for det in pred[0]:
        # Extract bounding box coordinates (x_center, y_center, width, height)
        x_center, y_center, width, height = det[:4]

        # Convert to (x1, y1, x2, y2) format
        x1 = int((x_center - width / 2) * (frame.shape[1] / 320))  # Scale to original frame size
        y1 = int((y_center - height / 2) * (frame.shape[0] / 320))
        x2 = int((x_center + width / 2) * (frame.shape[1] / 320))
        y2 = int((y_center + height / 2) * (frame.shape[0] / 320))

        # Extract confidence score
        conf = det[4]  # Confidence score is typically at index 4

        # Filter out low-confidence predictions
        if conf > CONF_THRESH:
            boxes.append([x1, y1, x2, y2])
            confidences.append(float(conf))

    # Apply Non-Maximum Suppression (NMS)
    indices = cv2.dnn.NMSBoxes(boxes, confidences, CONF_THRESH, NMS_THRESH)
    if indices is not None and len(indices) > 0:
        for i in indices.flatten():
            x1, y1, x2, y2 = boxes[i]
            conf = confidences[i]
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"Face {conf:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Show the frame
    cv2.imshow("YOLOv5 Face Detection", frame)

    # Press 'Esc' to exit
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()